# `TohuNamespaceNEW`


[TOC]


The `TohuNamespaceNEW` class allows grouping together other tohu generators and treating them as a single unit (which is used to implement the functionality of custom generators).

In [1]:
from tohu.tohu_namespace_NEW import TohuNamespaceNEW

## Initialisation and adding generators

In [2]:
from tohu import Integer, HashDigest, FakerGenerator

In [3]:
tohu_namespace = TohuNamespaceNEW()

tohu_namespace.add_field_generator("xx", Integer(100, 200))
tohu_namespace.add_field_generator("yy", HashDigest(length=6))
tohu_namespace.add_field_generator("zz", FakerGenerator(method="name"))

tohu_namespace.field_generators

{'xx': <Integer (id=4caa28)>,
 'yy': <HashDigest (id=c90feb)>,
 'zz': <FakerGenerator (id=e43716)>}

The convenience method `add_tohu_generators_from_dict()` allows passing a dictionary, and it will call `add_generator()` for any tohu generators found in this dictionary (while ignoring any other values).

In [4]:
generators = {
    "xx": Integer(100, 200),
    "yy": HashDigest(length=6),
    "aa": "this will not be added because it is not a tohu generator",
    "zz": FakerGenerator(method="name"),
    "bb": 42
}

In [5]:
tohu_namespace = TohuNamespaceNEW()
tohu_namespace.add_field_generators_from_dict(generators)

In [6]:
tohu_namespace.field_generators

{'xx': <Integer (id=39a8a3)>,
 'yy': <HashDigest (id=fc9854)>,
 'zz': <FakerGenerator (id=8bd761)>}

## Setting the `tohu_items_cls` attribute

Initially the `tohu_items_cls` attribute refers to a non-existent tohu items class:

In [7]:
tohu_namespace.tohu_items_cls

<NonExistentTohuItemsClass>

Once all desired generators have been added to the tohu namespace, we can call `set_tohu_items_class`, which will automatically create a tohu items class with the same field names as the generators contained in the namespace.

In [8]:
tohu_namespace.set_tohu_items_class(name="Quux")

In [9]:
tohu_namespace.tohu_items_cls

tohu.tohu_items_class.Quux

This items class can then be used to create individual tohu items.

In [10]:
tohu_namespace.tohu_items_cls(xx=100, yy="910A97", zz="Kristen Wallace")

Quux(xx=100, yy='910A97', zz='Kristen Wallace')

## Resetting and generating tohu items

In [11]:
tohu_namespace.field_generators

{'xx': <Integer (id=39a8a3)>,
 'yy': <HashDigest (id=fc9854)>,
 'zz': <FakerGenerator (id=8bd761)>}

In [12]:
tohu_namespace.reset(seed=11111)

print(next(tohu_namespace))
print(next(tohu_namespace))
print(next(tohu_namespace))
print(next(tohu_namespace))
print(next(tohu_namespace))

Quux(xx=163, yy='7551AA', zz='Michelle Miller')
Quux(xx=171, yy='54596E', zz='Eddie Davis')
Quux(xx=142, yy='2A16D0', zz='Kathleen Lucas')
Quux(xx=140, yy='FDCDD3', zz='Jason Rodriguez')
Quux(xx=121, yy='BDE283', zz='Andrew Pitts')


## Adding generators with dependencies

In [13]:
from tohu.derived_generators import Apply

In [14]:
aa = Integer(1, 9)
bb = Apply(lambda x: x*11, aa)
cc = Apply(lambda x: x*101, bb)

In [15]:
assert bb.arg_gens[0].is_clone_of(aa)
assert cc.arg_gens[0].is_clone_of(bb)

In [16]:
tohu_namespace = TohuNamespaceNEW()
tohu_namespace.add_field_generator("rr", aa)
tohu_namespace.add_field_generator("ss", aa)
tohu_namespace.add_field_generator("tt", bb)
tohu_namespace.add_field_generator("uu", cc)
tohu_namespace.add_field_generator("vv", cc)
tohu_namespace.set_tohu_items_class("Quux")

In [17]:
tohu_namespace.field_generators

{'rr': <Integer (id=4534c4)>,
 'ss': <Integer (id=f164aa)>,
 'tt': <Apply (id=ec19e1)>,
 'uu': <Apply (id=e1d522)>,
 'vv': <Apply (id=9ed21a)>}

Note that even though the generator `aa` is added to the namespace twice (first with the name `"rr"` and then with the name `"ss"`), `ss` actually ends up as a _clone_ of `rr` (this is to ensure that the tohu items produced by the namespace contain the correct values).

The following checks that this works as expected.

In [18]:
assert tohu_namespace.field_generators["ss"].is_clone_of(tohu_namespace.field_generators["rr"])
assert tohu_namespace.field_generators["tt"].arg_gens[0].is_clone_of(tohu_namespace.field_generators["rr"])
assert tohu_namespace.field_generators["uu"].arg_gens[0].is_clone_of(tohu_namespace.field_generators["tt"])
assert tohu_namespace.field_generators["vv"].parent.arg_gens[0].is_clone_of(tohu_namespace.field_generators["tt"])

tohu_namespace.reset(seed=11111)
print(next(tohu_namespace))
print(next(tohu_namespace))
print(next(tohu_namespace))
print(next(tohu_namespace))
print(next(tohu_namespace))

Quux(rr=8, ss=8, tt=88, uu=8888, vv=8888)
Quux(rr=9, ss=9, tt=99, uu=9999, vv=9999)
Quux(rr=6, ss=6, tt=66, uu=6666, vv=6666)
Quux(rr=6, ss=6, tt=66, uu=6666, vv=6666)
Quux(rr=3, ss=3, tt=33, uu=3333, vv=3333)


### Adding loop variables as "hidden" non-field generators

In [19]:
from tohu.looping_NEW import LoopVariableNEW

In [20]:
tohu_namespace = TohuNamespaceNEW()

xx = LoopVariableNEW("xx", values=[4, 5, 6])

tohu_namespace.add_loop_variable_as_hidden_non_field_generator("xx", xx)
tohu_namespace.add_field_generator("aa", HashDigest(length=6))
tohu_namespace.add_field_generator("bb", FakerGenerator(method="name"))
tohu_namespace.add_field_generator("cc", Apply(lambda x: 11111*x, xx))

In [21]:
tohu_namespace.all_generators

{'xx': <LoopVariable: name='xx', loop_level=None, values=[4, 5, 6], cur_value=4 (tohu_id=e11158)>,
 'aa': <HashDigest (id=4cc961)>,
 'bb': <FakerGenerator (id=d3f45f)>,
 'cc': <Apply (id=54ac33)>}

In [22]:
tohu_namespace.field_generators

{'aa': <HashDigest (id=4cc961)>,
 'bb': <FakerGenerator (id=d3f45f)>,
 'cc': <Apply (id=54ac33)>}

Note that when generating items, only 

In [23]:
tohu_namespace.set_tohu_items_class("Quux")
tohu_namespace.reset_loop_variables()

tohu_namespace.reset(seed=11111)
print(next(tohu_namespace))
print(next(tohu_namespace))
tohu_namespace.all_generators["xx"].advance()
print(next(tohu_namespace))
print(next(tohu_namespace))

Quux(aa='4888AF', bb='Pamela Roman', cc=44444)
Quux(aa='FFFE41', bb='Timothy Mendoza', cc=44444)
Quux(aa='096A60', bb='David Zamora', cc=55555)
Quux(aa='4CFAE9', bb='Michael King', cc=55555)


Note that the following call to `reset()` does not reset the loop variable (and thus also doesn't reset the field generator `cc`, which continues with the previous value).

In [24]:
tohu_namespace.reset(seed=22222)

print(next(tohu_namespace))
print(next(tohu_namespace))
tohu_namespace.all_generators["xx"].advance()
print(next(tohu_namespace))
print(next(tohu_namespace))

Quux(aa='46F342', bb='Adam Brown', cc=55555)
Quux(aa='06E2E4', bb='Rebecca Mcdonald', cc=55555)
Quux(aa='215A77', bb='Tanya Dickerson', cc=66666)
Quux(aa='880E2A', bb='Joseph Freeman', cc=66666)


### Extracting loop variables from a tohu namespace into a loop runner

The method `extract_loop_runner()` finds all loop variables in the tohu namespace and creates a `LoopRunner` instance containing them. This is used internally by custom generators to orchestrate looping.

In [39]:
tohu_namespace = TohuNamespaceNEW()

xx = LoopVariableNEW("xx", values=[4, 5, 6]).set_loop_level(42)
yy = LoopVariableNEW("yy", values=["AAA", "BBB"]).set_loop_level(23)

tohu_namespace.add_loop_variable_as_hidden_non_field_generator("xx", xx)
tohu_namespace.add_field_generator("aa", HashDigest(length=6))
tohu_namespace.add_field_generator("bb", FakerGenerator(method="name"))
tohu_namespace.add_loop_variable_as_hidden_non_field_generator("yy", yy)
tohu_namespace.add_field_generator("cc", Apply(lambda x, y: x + y, xx, yy))

loop_runner = tohu_namespace.extract_loop_runner()

In [40]:
loop_runner.loop_variables

{'xx': <LoopVariable: name='xx', loop_level=42, values=[4, 5, 6], cur_value=4 (tohu_id=f823f9)>,
 'yy': <LoopVariable: name='yy', loop_level=23, values=['AAA', 'BBB'], cur_value='AAA' (tohu_id=c08f80)>}

In [41]:
loop_runner.max_loop_level

42

**TODO:**

Add tests for:
- [X] `.set_tohu_items_class("Quux")`
- [X] `.reset()`
- [X] `.__next__()`
- [X] adding generators with dependencies on previously added generators (check that "rewiring" works correctly)
- [X] adding loop variables (these should end up as hidden generators, not as publicly visible field generators)
- [X] ensure that when `.reset()` is called, only the expected generators are reset (in particular, no clones; and loop variables are properly reset too)
- [X] `.extract_loop_runner()`

(and any other usages in the notebook `2020-04-08__Prototyping_TohuNamespace_and_LoopRunner_v2.ipynb`)